In [1]:
import numpy as np
import pandas as pd

from selenium import webdriver as wb
from bs4 import BeautifulSoup as bs

from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

import time 
from tqdm import tqdm

import requests
from datetime import datetime
import json
import pickle

## 데이터 불러오기

In [2]:
df = pd.read_csv("data/request_reviews.csv")
df.head(3)

,Unnamed: 0,MGTNO,APVPERMYMD,TRDSTATENM,SITEAREA,SITEPOSTNO,SITEWHLADDR,RDNWHLADDR,BPLCNM,UPTAENM,X,Y,TRDPJUBNSENM,WTRSPLYFACILSENM,dong,exemplary_diner,keyword,id,id_mod,id_mod2
0,0,3220000-101-1007-00001,19970205,영업/정상,NaN,135954,서울특별시 강남구 청담동 88-5번지,서울 강남구 도산대로55길 20,배초향,한식,203520.881091,446950.285626,NaN,NaN,청담동,NaN,배초향,NaN,,
1,1,3220000-101-1899-20451,19991005,영업/정상,950.66,135090,서울특별시 강남구 삼성동 159,서울 강남구 봉은사로 524,브래서리,경양식,205130.591679,445590.096838,기타,상수도전용,삼성동,모범식당지정,브래서리,['10825881'],10825881,10825881
2,2,3220000-101-1899-20453,19991005,영업/정상,473.24,135090,서울특별시 강남구 삼성동 159,서울 강남구 봉은사로 524,로비라운지,경양식,205130.591679,445590.096838,기타,상수도전용,삼성동,NaN,로비라운지,['727661240'],727661240,727661240


In [3]:
df.shape

(12528, 20)

In [4]:
df[df.keyword.isna()]

,Unnamed: 0,MGTNO,APVPERMYMD,TRDSTATENM,SITEAREA,SITEPOSTNO,SITEWHLADDR,RDNWHLADDR,BPLCNM,UPTAENM,X,Y,TRDPJUBNSENM,WTRSPLYFACILSENM,dong,exemplary_diner,keyword,id,id_mod,id_mod2
247,247,3220000-101-1990-07507,19901101,영업/정상,31.73,135860,서울특별시 강남구 도곡동 954-19번지,서울 강남구 남부순환로355길 13,(신)오양절면,한식,203087.887972,442628.722914,기타,상수도전용,도곡동,NaN,NaN,['338342608'],NaN,NaN
994,994,3220000-101-1998-12691,19981029,영업/정상,94.63,135515,서울특별시 강남구 역삼동 787-1번지,서울 강남구 논현로64길 22,(중화요리)남경,중국식,203668.426416,443485.604196,기타,상수도전용,역삼동,NaN,NaN,['10960059'],NaN,NaN
1432,1432,3220000-101-2000-22499,2000-09-27,영업/정상,677.61,135-731,서울특별시 강남구 삼성동 159 코엑스,서울 강남구 영동대로 513,(유)아웃백스테이크하우스코리아 코엑스스타필드점,경양식,205130.591679,445590.096838,NaN,NaN,삼성동,NaN,NaN,['16877892'],NaN,NaN
3866,3866,3220000-101-2011-00114,20110223,영업/정상,81.78,135955,서울특별시 강남구 청담동 122-7 1층,서울 강남구 도산대로85길 14,(첼시스 하이볼) Chelsea's HIGHBALL,일식,204279.144297,447023.007955,NaN,NaN,청담동,NaN,NaN,['19153490'],NaN,NaN
4031,4031,3220000-101-2011-00686,20110811,영업/정상,49.50,135515,서울특별시 강남구 역삼동 781-41 지상1층102호,서울 강남구 논현로66길 9,(SOS)커피 역삼점,경양식,203566.263883,443554.896146,NaN,NaN,역삼동,NaN,NaN,['14600986'],NaN,NaN
4511,4511,3220000-101-2013-00033,20130117,영업/정상,177.28,135911,서울특별시 강남구 역삼동 646-4번지,서울 강남구 강남대로94길 86,(이층) 맥주창고,경양식,202951.131852,444272.491202,NaN,NaN,역삼동,NaN,NaN,NaN,NaN,NaN
6054,6054,3220000-101-2016-00899,2016-10-21,영업/정상,507.39,135-825,서울특별시 강남구 논현동 165,서울 강남구 강남대로 502,(유)아웃백스테이크하우스코리아 강남점,경양식,202024.615630,444994.659485,NaN,NaN,논현동,NaN,NaN,['522500805'],NaN,NaN
8697,8697,3220000-101-2020-01279,20201030,영업/정상,268.98,135896,서울특별시 강남구 신사동 637-11,서울 강남구 압구정로42길 12,(끌레오)CLO,한식,203049.750409,447323.452649,NaN,NaN,신사동,NaN,NaN,['1495236287'],NaN,NaN
9148,9148,3220000-101-2021-00506,20210503,영업/정상,33.00,135949,서울특별시 강남구 청담동 9-12,서울 강남구 학동로77길 50,(유노시엘) EUNOSIERE,기타,204171.424581,446636.433530,NaN,NaN,청담동,NaN,NaN,['118575691'],NaN,NaN
9354,9354,3220000-101-2021-00858,20210708,영업/정상,159.25,135915,서울특별시 강남구 역삼동 676 센터필드,서울 강남구 테헤란로 231,(유)한국집 센터필드 역삼점,한식,203599.062772,444556.062267,NaN,NaN,역삼동,NaN,NaN,['1872560550'],NaN,NaN


In [5]:
df[df.keyword=="양문"]

,Unnamed: 0,MGTNO,APVPERMYMD,TRDSTATENM,SITEAREA,SITEPOSTNO,SITEWHLADDR,RDNWHLADDR,BPLCNM,UPTAENM,X,Y,TRDPJUBNSENM,WTRSPLYFACILSENM,dong,exemplary_diner,keyword,id,id_mod,id_mod2
248,248,3220000-101-1990-07519,1990-06-22,영업/정상,97.90,135-841,서울특별시 강남구 대치동 906,서울 강남구 선릉로76길 24,양문,한식,204574.569178,444393.323912,NaN,NaN,대치동,NaN,양문,['912331600'],338342608,338342608
5695,5695,3220000-101-2015-01126,2015-12-09,영업/정상,171.60,135-915,서울특별시 강남구 역삼동 677-5,서울 강남구 언주로93길 21,양문(역삼점),기타,203495.929163,444569.786892,NaN,NaN,역삼동,모범식당지정,양문,NaN,,1054657548
8587,8587,3220000-101-2020-01054,20200911,영업/정상,87.30,135888,서울특별시 강남구 신사동 525-24,서울 강남구 압구정로4길 13-17,양문,감성주점,201827.044357,446696.265274,NaN,NaN,신사동,NaN,양문,['1628381325'],234893288,234893288


In [6]:
# df["url"] = df["id_mod2"].apply(lambda x: np.nan if x==" " else x)
df["url"] = df["id_mod2"].fillna(" ")
df["url"] = df["url"].apply(lambda x: "https://place.map.kakao.com/" + x if x!=" " else x)

In [7]:
df["keyword2"] = df["keyword"].fillna(" ")

# 업소 이름 뒤에 위치한 동 이름 붙임
def plus_dong(txt):
    if txt["dong"][:-1] not in txt["keyword2"]:
        txt["name"] = txt["keyword2"] + " " + txt["dong"][:-1]
    else:
        txt["name"] = txt["keyword2"]
    return txt

In [8]:
# plus_dong 함수 적용
df = df.apply(plus_dong, axis=1)

## request 적용 -> 실패

In [9]:
df.head(3)

,Unnamed: 0,MGTNO,APVPERMYMD,TRDSTATENM,SITEAREA,SITEPOSTNO,SITEWHLADDR,RDNWHLADDR,BPLCNM,UPTAENM,...,WTRSPLYFACILSENM,dong,exemplary_diner,keyword,id,id_mod,id_mod2,url,keyword2,name
0,0,3220000-101-1007-00001,19970205,영업/정상,NaN,135954,서울특별시 강남구 청담동 88-5번지,서울 강남구 도산대로55길 20,배초향,한식,...,NaN,청담동,NaN,배초향,NaN,,,,배초향,배초향 청담
1,1,3220000-101-1899-20451,19991005,영업/정상,950.66,135090,서울특별시 강남구 삼성동 159,서울 강남구 봉은사로 524,브래서리,경양식,...,상수도전용,삼성동,모범식당지정,브래서리,['10825881'],10825881,10825881,https://place.map.kakao.com/10825881,브래서리,브래서리 삼성
2,2,3220000-101-1899-20453,19991005,영업/정상,473.24,135090,서울특별시 강남구 삼성동 159,서울 강남구 봉은사로 524,로비라운지,경양식,...,상수도전용,삼성동,NaN,로비라운지,['727661240'],727661240,727661240,https://place.map.kakao.com/727661240,로비라운지,로비라운지 삼성


In [10]:
df[df.id_mod2 != " "].shape


(7289, 23)

In [11]:
df["id_mod2"][:3]

0             
1     10825881
2    727661240
Name: id_mod2, dtype: object

In [11]:
request_id = 14569639
url = f'https://place.map.kakao.com/main/v/{request_id}'
response = requests.get(url)
comments = []
if response.status_code == 200:
    comment_data = json.loads(response.content)['comment']
    comment_count = int(comment_data['kamapComntcnt'])
    overall_score = int(comment_data['scoresum']) / int(comment_data['scorecnt'])
    for c in comment_data['list']:
        comments.append({'점수': c['point'], '댓글': c['contents'], '작성일': c['date']})
        request_comment_id = c['commentid']
    print(f'총 댓글 개수: {comment_count}개, 평점: {np.round(overall_score, 2)}점')
else:
    print('error')

총 댓글 개수: 169개, 평점: 4.11점


In [12]:
while len(comments) < comment_count:
    url = f'https://place.map.kakao.com/commentlist/v/{request_id}/{request_comment_id}'
    response = requests.get(url)
    if response.status_code == 200:
        comment_data = json.loads(response.content)['comment']
        for c in comment_data['list']:
            try:
                comments.append({'점수': c['point'], '댓글': c['contents'], '작성일': c['date']})
            except:
                comments.append({'점수': c['point'], '댓글': '', '작성일': c['date']})
            request_comment_id = c['commentid']
    else:
        print('error')

In [13]:
pd.DataFrame(comments)

,점수,댓글,작성일
0,5,,2024.04.22.
1,2,너무 짜요. 처음엔 먹을만했는데. 절반정도 먹고나선 너무 짜서 짜증날 정도임. 맛 ...,2024.04.14.
2,5,곰탕 정말 빨리나옵니다. 주문하고 5분도 안되서 음식 받은거 같아요. 정말 맛있고 ...,2024.04.11.
3,5,수육이랑 곰탕먹었어요\n곰탕은 주문하자마자 빨리 나왔어요\n고기도 많이 들어가있고\...,2024.04.08.
4,5,세종대왕의 이름 (이도 李裪)에 걸맞는 \n진한 맛의 곰탕입니다\n마침 제옆에 외국...,2024.04.02.
...,...,...,...
164,4,곰탕,2016.09.29.
165,4,소수육46. 면+죽 추가10.,2016.09.09.
166,2,공장형.\n밥대신 국수.\n잘 모르겠다.,2016.08.03.
167,4,,2016.07.10.


In [14]:
comment_data

{'kamapComntcnt': 169,
 'scoresum': 695,
 'scorecnt': 169,
 'list': [{'commentid': '842665',
   'contents': '(수육+장칼국수+죽) 3명에서 이렇게 먹으니 딱좋네.\n복잡한 역삼역에 주차공간도 나름 넉넉 (10대정도?)\n다음에는 곰탕을 먹어 봐야겠다.',
   'point': 5,
   'username': '자유로',
   'profile': 'http://t1.daumcdn.net/local/kakaomapPhoto/profile/381d738b852dead3074f44ea9effa8bf24a6f1bb?original',
   'profileStatus': 'S',
   'photoCnt': 1,
   'likeCnt': 0,
   'thumbnail': 'https://t1.daumcdn.net/local/review_placeapp/oWaiP3LwQf_QimK4eF87DMjZwTKIiln20_lkagebjc44A.jpeg',
   'kakaoMapUserId': '267318248',
   'photoList': [{'url': 'https://t1.daumcdn.net/local/review_placeapp/oWaiP3LwQf_QimK4eF87DMjZwTKIiln20_lkagebjc44A.jpeg',
     'near': False}],
   'ownerReply': {},
   'userCommentCount': 72,
   'userCommentAverageScore': 4.2,
   'myStorePick': False,
   'level': {'nowLevel': 28, 'badge': '02'},
   'date': '2016.03.15.',
   'isMy': False,
   'isBlock': False,
   'isEditable': False,
   'isMyLike': False}],
 'strengthCounts': [{'id': 5, 'nam

In [12]:
df["id_mod2"].shape[0]/3, df["id_mod2"].shape[0]/3*2, df["id_mod2"].shape[0]/3*3

(4176.0, 8352.0, 12528.0)

In [11]:
comments = []

for i, id in enumerate(df["id_mod2"][8352:], 8352):
# for i, id in enumerate(df["id_mod2"][8352:8355], 8352):
    
    if id==" ":
        continue
    else:
        try:
            time.sleep(1)
            request_id = id
            url = f'https://place.map.kakao.com/main/v/{request_id}'
            response = requests.get(url)
            # print(url)

            try:

                if response.status_code == 200:
                    comment_num = 3
                    comment_data = json.loads(response.content)['comment']
                    comment_count = int(comment_data['kamapComntcnt'])
                    overall_score = int(comment_data['scoresum']) / int(comment_data['scorecnt'])
                    for c in comment_data['list']:
                        # comments.append({'점수': c['point'], '댓글': c['contents'], '작성일': c['date']})
                        comments.append({"name": df.loc[i, "keyword"], "user": c["username"], 
                                                'review': c['contents'], 'date': c['date'],
                                                'rate': c['point'], "avg_rate": c["userCommentAverageScore"],
                                                "review_cnt": c["userCommentCount"], "is_model": df.loc[i, "exemplary_diner"]})
                        request_comment_id = c['commentid']
                    
                    while comment_num < comment_count:
                        url = f'https://place.map.kakao.com/commentlist/v/{request_id}/{request_comment_id}'
                        response = requests.get(url)
                        if response.status_code == 200:
                            comment_data = json.loads(response.content)['comment']
                            for c in comment_data['list']:
                                comment_num += 1
                                try:
                                    comments.append({"name": df.loc[i, "keyword"], "user": c["username"], 
                                                    'review': c['contents'], 'date': c['date'],
                                                    'rate': c['point'], "avg_rate": c["userCommentAverageScore"],
                                                    "review_cnt": c["userCommentCount"], "is_model": df.loc[i, "exemplary_diner"]})
                                except:
                                    comments.append({"name": df.loc[i, "keyword"], "user": c["username"], 
                                                    'review': "", 'date': c['date'],
                                                    'rate': c['point'], "avg_rate": c["userCommentAverageScore"],
                                                    "review_cnt": c["userCommentCount"], "is_model": df.loc[i, "exemplary_diner"]})
                                request_comment_id = c['commentid']
                            # print(f'총 댓글 개수: {comment_count}개, 평점: {np.round(overall_score, 2)}점')
                else:
                    print('response')
                    # pass
            except:
                print('comments error')
        
        except:
            print("url_error")
            
        

comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
response
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
c

In [12]:
pd.DataFrame(comments)

,name,user,review,date,rate,avg_rate,review_cnt,is_model
0,부산아지매국밥 삼성중앙,psqa,타지점에 비해 국물이 밍밍함. 김치제공x. 직장인들 많이옴.,2024.03.07.,3,3.1,30,NaN
1,부산아지매국밥 삼성중앙,서윤,체인점 이지만 어디든 맛있는 부산 아지매 국밥!,2023.06.16.,4,4.0,41,NaN
2,부산아지매국밥 삼성중앙,나혜,저 원래 리뷰 잘 안 남기는데\n일단 밥이 빨리빨리 나오고요\n깍두기도 슴슴하니 맛...,2023.03.15.,5,5.0,2,NaN
3,부산아지매국밥 삼성중앙,오더리움,맛있어요,2023.02.22.,5,3.7,32,NaN
4,부산아지매국밥 삼성중앙,캐주얼큐,제 입맛은 아니었어요. 성의 없이 딱딱하고 맛없는 깍두기와 퍽퍽한 돼지고기,2022.04.12.,1,3.8,250,NaN
...,...,...,...,...,...,...,...,...
22605,징기스칸 슌,s,개인 프라이빗룸도 있고 카운터형으로 직원분들이 다 구워주셔서 거래처 식사 대접할때나...,2024.03.29.,5,5.0,1,NaN
22606,월출산,대방어,,2024.04.23.,1,3.5,489,NaN
22607,월출산,김지성,"mild seaweed soup, a bit hard and cold rice, o...",2024.04.11.,4,4.3,7,NaN
22608,월출산,서기,사장님! 직원교육좀시키세요 제발 밥먹으러갈때마다 눈치보이고 체할거같아요 너무 불친절...,2023.07.25.,1,1.7,11,NaN


In [14]:
df[df.keyword=="월출산"]

,Unnamed: 0,MGTNO,APVPERMYMD,TRDSTATENM,SITEAREA,SITEPOSTNO,SITEWHLADDR,RDNWHLADDR,BPLCNM,UPTAENM,...,WTRSPLYFACILSENM,dong,exemplary_diner,keyword,id,id_mod,id_mod2,url,keyword2,name
5878,5878,3220000-101-2016-00412,20160520,영업/정상,145.00,135881,서울특별시 강남구 삼성동 162-15,서울 강남구 봉은사로112길 6,월출산,한식,...,NaN,삼성동,모범식당지정,월출산,NaN,,107246805,https://place.map.kakao.com/107246805,월출산,월출산 삼성
12352,12352,3220000-101-2024-00311,2024-03-14,영업/정상,303.00,135-881,서울특별시 강남구 삼성동 165-11 상원빌딩,서울 강남구 봉은사로112길 22,월출산,한식,...,NaN,삼성동,NaN,월출산,NaN,,107246805,https://place.map.kakao.com/107246805,월출산,월출산 삼성


In [15]:
reviews_8352 = pd.DataFrame(comments)
reviews_8352

,name,user,review,date,rate,avg_rate,review_cnt,is_model
0,부산아지매국밥 삼성중앙,psqa,타지점에 비해 국물이 밍밍함. 김치제공x. 직장인들 많이옴.,2024.03.07.,3,3.1,30,NaN
1,부산아지매국밥 삼성중앙,서윤,체인점 이지만 어디든 맛있는 부산 아지매 국밥!,2023.06.16.,4,4.0,41,NaN
2,부산아지매국밥 삼성중앙,나혜,저 원래 리뷰 잘 안 남기는데\n일단 밥이 빨리빨리 나오고요\n깍두기도 슴슴하니 맛...,2023.03.15.,5,5.0,2,NaN
3,부산아지매국밥 삼성중앙,오더리움,맛있어요,2023.02.22.,5,3.7,32,NaN
4,부산아지매국밥 삼성중앙,캐주얼큐,제 입맛은 아니었어요. 성의 없이 딱딱하고 맛없는 깍두기와 퍽퍽한 돼지고기,2022.04.12.,1,3.8,250,NaN
...,...,...,...,...,...,...,...,...
22605,징기스칸 슌,s,개인 프라이빗룸도 있고 카운터형으로 직원분들이 다 구워주셔서 거래처 식사 대접할때나...,2024.03.29.,5,5.0,1,NaN
22606,월출산,대방어,,2024.04.23.,1,3.5,489,NaN
22607,월출산,김지성,"mild seaweed soup, a bit hard and cold rice, o...",2024.04.11.,4,4.3,7,NaN
22608,월출산,서기,사장님! 직원교육좀시키세요 제발 밥먹으러갈때마다 눈치보이고 체할거같아요 너무 불친절...,2023.07.25.,1,1.7,11,NaN


In [16]:
reviews_8352.name.nunique()

1813

In [29]:
df[df["keyword"]=="월출산"]

,Unnamed: 0,MGTNO,APVPERMYMD,TRDSTATENM,SITEAREA,SITEPOSTNO,SITEWHLADDR,RDNWHLADDR,BPLCNM,UPTAENM,...,WTRSPLYFACILSENM,dong,exemplary_diner,keyword,id,id_mod,id_mod2,url,keyword2,name
5878,5878,3220000-101-2016-00412,20160520,영업/정상,145.00,135881,서울특별시 강남구 삼성동 162-15,서울 강남구 봉은사로112길 6,월출산,한식,...,NaN,삼성동,모범식당지정,월출산,NaN,,107246805,https://place.map.kakao.com/107246805,월출산,월출산 삼성
12352,12352,3220000-101-2024-00311,2024-03-14,영업/정상,303.00,135-881,서울특별시 강남구 삼성동 165-11 상원빌딩,서울 강남구 봉은사로112길 22,월출산,한식,...,NaN,삼성동,NaN,월출산,NaN,,107246805,https://place.map.kakao.com/107246805,월출산,월출산 삼성


In [20]:
df[df["keyword"].fillna("").str.contains("베러디")]


,Unnamed: 0,MGTNO,APVPERMYMD,TRDSTATENM,SITEAREA,SITEPOSTNO,SITEWHLADDR,RDNWHLADDR,BPLCNM,UPTAENM,...,WTRSPLYFACILSENM,dong,exemplary_diner,keyword,id,id_mod,id_mod2,url,keyword2,name
8359,8359,3220000-101-2020-00622,20200611,영업/정상,61.30,135811,서울특별시 강남구 논현동 12번지,서울 강남구 논현로149길 65,베러디(betterd),기타,...,NaN,논현동,NaN,베러디,NaN,365061365,365061365,https://place.map.kakao.com/365061365,베러디,베러디 논현


In [21]:
df[df["keyword"]=="제주흑돼지마을"]

,Unnamed: 0,MGTNO,APVPERMYMD,TRDSTATENM,SITEAREA,SITEPOSTNO,SITEWHLADDR,RDNWHLADDR,BPLCNM,UPTAENM,...,WTRSPLYFACILSENM,dong,exemplary_diner,keyword,id,id_mod,id_mod2,url,keyword2,name
8352,8352,3220000-101-2020-00608,2020-06-09,영업/정상,97.55,135-833,서울특별시 강남구 논현동 277-31 내외빌딩,서울 강남구 봉은사로 319,제주흑돼지마을,한식,...,NaN,논현동,NaN,제주흑돼지마을,['365061365'],,,,제주흑돼지마을,제주흑돼지마을 논현


In [22]:
df.shape

(12528, 23)

In [23]:
# index 8352 부터 수집한 리뷰 데이터 csv 저장 
reviews_8352.to_csv("data/reviews_8352.csv", index=False, encoding="utf-8")


In [13]:
comments = []

for i, id in enumerate(df["id_mod2"][8352:], 8352):
# for i, id in enumerate(df["id_mod2"][8352:8355], 8352):
    
    if id==" ":
        continue
    else:
        try:
            time.sleep(1)
            request_id = id
            url = f'https://place.map.kakao.com/main/v/{request_id}'
            response = requests.get(url)
            # print(url)

            try:

                if response.status_code == 200:
                    comment_num = 3
                    comment_data = json.loads(response.content)['comment']
                    comment_count = int(comment_data['kamapComntcnt'])
                    overall_score = int(comment_data['scoresum']) / int(comment_data['scorecnt'])
                    for c in comment_data['list']:

                        try:
                            user = c["username"]
                        except:
                            user = ''

                        try:
                            review = c['contents']
                        except:
                            review = ''

                        # comments.append({'점수': c['point'], '댓글': c['contents'], '작성일': c['date']})
                        comments.append({"name": df.loc[i, "keyword"], "user": c["username"], 
                                                'review': c['contents'], 'date': c['date'],
                                                'rate': c['point'], "avg_rate": c["userCommentAverageScore"],
                                                "review_cnt": c["userCommentCount"], "is_model": df.loc[i, "exemplary_diner"]})
                        request_comment_id = c['commentid']
                    
                    while comment_num < comment_count:
                        url = f'https://place.map.kakao.com/commentlist/v/{request_id}/{request_comment_id}'
                        response = requests.get(url)
                        if response.status_code == 200:
                            comment_data = json.loads(response.content)['comment']
                            for c in comment_data['list']:

                                try:
                                    user = c["username"]
                                except:
                                    user = ''

                                try:
                                    review = c['contents']
                                except:
                                    review = ''

                                comment_num += 1
                                try:
                                    comments.append({"name": df.loc[i, "keyword"], "user": c["username"], 
                                                    'review': c['contents'], 'date': c['date'],
                                                    'rate': c['point'], "avg_rate": c["userCommentAverageScore"],
                                                    "review_cnt": c["userCommentCount"], "is_model": df.loc[i, "exemplary_diner"]})
                                except:
                                    comments.append({"name": df.loc[i, "keyword"], "user": c["username"], 
                                                    'review': "", 'date': c['date'],
                                                    'rate': c['point'], "avg_rate": c["userCommentAverageScore"],
                                                    "review_cnt": c["userCommentCount"], "is_model": df.loc[i, "exemplary_diner"]})
                                request_comment_id = c['commentid']
                            # print(f'총 댓글 개수: {comment_count}개, 평점: {np.round(overall_score, 2)}점')
                else:
                    print('response')
                    # pass
            except:
                print('comments error')
        
        except:
            print("url_error")
            
        

comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
response
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
c

In [26]:
df[df.keyword=="펀비어킹"]

,Unnamed: 0,MGTNO,APVPERMYMD,TRDSTATENM,SITEAREA,SITEPOSTNO,SITEWHLADDR,RDNWHLADDR,BPLCNM,UPTAENM,...,WTRSPLYFACILSENM,dong,exemplary_diner,keyword,id,id_mod,id_mod2,url,keyword2,name
2571,2571,3220000-101-2006-00134,20060308,영업/정상,148.68,135916,"서울특별시 강남구 역삼동 681-3번지 지상1, 2층",서울 강남구 언주로108길 15,펀비어킹,통닭(치킨),...,상수도전용,역삼동,NaN,펀비어킹,['16534083'],26647710,26647710,https://place.map.kakao.com/26647710,펀비어킹,펀비어킹 역삼
3527,3527,3220000-101-2010-00012,20100108,영업/정상,317.73,135860,서울특별시 강남구 도곡동 953-3번지 지상2층,서울 강남구 강남대로 242,펀비어킹,패스트푸드,...,상수도전용,도곡동,NaN,펀비어킹,['15945851'],,,,펀비어킹,펀비어킹 도곡


In [17]:
reviews_collect = pd.DataFrame(comments)
reviews_collect

,name,user,review,date,rate,avg_rate,review_cnt,is_model
0,부산아지매국밥 삼성중앙,psqa,타지점에 비해 국물이 밍밍함. 김치제공x. 직장인들 많이옴.,2024.03.07.,3,3.1,30,NaN
1,부산아지매국밥 삼성중앙,서윤,체인점 이지만 어디든 맛있는 부산 아지매 국밥!,2023.06.16.,4,4.0,41,NaN
2,부산아지매국밥 삼성중앙,나혜,저 원래 리뷰 잘 안 남기는데\n일단 밥이 빨리빨리 나오고요\n깍두기도 슴슴하니 맛...,2023.03.15.,5,5.0,2,NaN
3,부산아지매국밥 삼성중앙,오더리움,맛있어요,2023.02.22.,5,3.7,32,NaN
4,부산아지매국밥 삼성중앙,캐주얼큐,제 입맛은 아니었어요. 성의 없이 딱딱하고 맛없는 깍두기와 퍽퍽한 돼지고기,2022.04.12.,1,3.8,250,NaN
...,...,...,...,...,...,...,...,...
22643,징기스칸 슌,s,개인 프라이빗룸도 있고 카운터형으로 직원분들이 다 구워주셔서 거래처 식사 대접할때나...,2024.03.29.,5,5.0,1,NaN
22644,월출산,대방어,,2024.04.23.,1,3.5,489,NaN
22645,월출산,김지성,"mild seaweed soup, a bit hard and cold rice, o...",2024.04.11.,4,4.3,7,NaN
22646,월출산,서기,사장님! 직원교육좀시키세요 제발 밥먹으러갈때마다 눈치보이고 체할거같아요 너무 불친절...,2023.07.25.,1,1.7,11,NaN


In [18]:
reviews_collect['is_model_b'] = reviews_collect['is_model'].map({np.nan: 0, '모범식당지정': 1})

In [19]:
reviews_collect

,name,user,review,date,rate,avg_rate,review_cnt,is_model,is_model_b
0,부산아지매국밥 삼성중앙,psqa,타지점에 비해 국물이 밍밍함. 김치제공x. 직장인들 많이옴.,2024.03.07.,3,3.1,30,NaN,0
1,부산아지매국밥 삼성중앙,서윤,체인점 이지만 어디든 맛있는 부산 아지매 국밥!,2023.06.16.,4,4.0,41,NaN,0
2,부산아지매국밥 삼성중앙,나혜,저 원래 리뷰 잘 안 남기는데\n일단 밥이 빨리빨리 나오고요\n깍두기도 슴슴하니 맛...,2023.03.15.,5,5.0,2,NaN,0
3,부산아지매국밥 삼성중앙,오더리움,맛있어요,2023.02.22.,5,3.7,32,NaN,0
4,부산아지매국밥 삼성중앙,캐주얼큐,제 입맛은 아니었어요. 성의 없이 딱딱하고 맛없는 깍두기와 퍽퍽한 돼지고기,2022.04.12.,1,3.8,250,NaN,0
...,...,...,...,...,...,...,...,...,...
22643,징기스칸 슌,s,개인 프라이빗룸도 있고 카운터형으로 직원분들이 다 구워주셔서 거래처 식사 대접할때나...,2024.03.29.,5,5.0,1,NaN,0
22644,월출산,대방어,,2024.04.23.,1,3.5,489,NaN,0
22645,월출산,김지성,"mild seaweed soup, a bit hard and cold rice, o...",2024.04.11.,4,4.3,7,NaN,0
22646,월출산,서기,사장님! 직원교육좀시키세요 제발 밥먹으러갈때마다 눈치보이고 체할거같아요 너무 불친절...,2023.07.25.,1,1.7,11,NaN,0


In [20]:
# index 8352 부터 수집한 리뷰 데이터 csv 저장 
reviews_collect.to_csv("data/reviews_collect.csv", index=False, encoding="utf-8")

In [24]:
reviews_collect.name.nunique()

1813

In [29]:
# 차마 못한 모범음식점 확인
last_mobeom = df.loc[3527:4176][df.loc[3527:4176].exemplary_diner.notnull()]
last_mobeom.shape

(26, 23)

In [30]:
last_mobeom

,Unnamed: 0,MGTNO,APVPERMYMD,TRDSTATENM,SITEAREA,SITEPOSTNO,SITEWHLADDR,RDNWHLADDR,BPLCNM,UPTAENM,...,WTRSPLYFACILSENM,dong,exemplary_diner,keyword,id,id_mod,id_mod2,url,keyword2,name
3546,3546,3220000-101-2010-00086,20100212,영업/정상,192.24,135897,서울특별시 강남구 신사동 663-7 지상2층,서울 강남구 선릉로157길 20-5,봉추찜닭압구정로데오점,한식,...,NaN,신사동,모범식당지정,봉추찜닭압구정로데오,['1497525458'],12543930,12543930,https://place.map.kakao.com/12543930,봉추찜닭압구정로데오,봉추찜닭압구정로데오 신사
3572,3572,3220000-101-2010-00195,2010-03-17,영업/정상,93.05,135-920,서울특별시 강남구 역삼동 720-25 1층역삼디오빌112호,서울 강남구 언주로 427,명동칼국수샤브샤브,한식,...,NaN,역삼동,모범식당지정,명동칼국수샤브샤브,['21332403'],12572463,12572463,https://place.map.kakao.com/12572463,명동칼국수샤브샤브,명동칼국수샤브샤브 역삼
3577,3577,3220000-101-2010-00211,20100323,영업/정상,281.70,135909,서울특별시 강남구 역삼동 641-20번지 지하1층,서울 강남구 논현로 511,통영집,한식,...,상수도전용,역삼동,모범식당지정,통영집,NaN,16062979,27256514,https://place.map.kakao.com/27256514,통영집,통영집 역삼
3652,3652,3220000-101-2010-00500,2010-06-10,영업/정상,405.33,135-500,서울특별시 강남구 대치동 1024-4 강원빌딩 지상1층,서울 강남구 도곡로 418,한티옥,한식,...,NaN,대치동,모범식당지정,한티옥,['1237681120'],1466449848,1466449848,https://place.map.kakao.com/1466449848,한티옥,한티옥 대치
3666,3666,3220000-101-2010-00551,20100622,영업/정상,297.75,135845,서울특별시 강남구 대치동 942-5번지 지상2층,서울 강남구 삼성로86길 7,용수사,일식,...,NaN,대치동,모범식당지정,용수사,['21311108'],8022618,8022618,https://place.map.kakao.com/8022618,용수사,용수사 대치
3740,3740,3220000-101-2010-00821,20100914,영업/정상,483.93,135933,서울특별시 강남구 역삼동 822-4번지 강남제일빌딩 1층,서울 강남구 테헤란로 109,갓텐스시강남점,일식,...,상수도전용,역삼동,모범식당지정,갓텐스시강남,NaN,13575898,13575898,https://place.map.kakao.com/13575898,갓텐스시강남,갓텐스시강남 역삼
3777,3777,3220000-101-2010-00937,20101021,영업/정상,102.47,135516,서울특별시 강남구 일원동 685-7번지 지상1층,서울 강남구 양재대로27길 6,신의주찹쌀순대,한식,...,상수도전용,일원동,모범식당지정,신의주찹쌀순대,['1528880295'],16080144,16080144,https://place.map.kakao.com/16080144,신의주찹쌀순대,신의주찹쌀순대 일원
3789,3789,3220000-101-2010-00965,20101028,영업/정상,484.64,135921,서울특별시 강남구 역삼동 725-9번지 지상1층,서울 강남구 논현로86길 20,베이징코야강남점,중국식,...,상수도전용,역삼동,모범식당지정,베이징코야강남,NaN,,NaN,,베이징코야강남,베이징코야강남 역삼
3829,3829,3220000-101-2010-01102,2010-12-09,영업/정상,209.99,135-875,서울특별시 강남구 삼성동 140 지상2층,서울 강남구 선릉로 530,신촌설렁탕,한식,...,NaN,삼성동,모범식당지정,신촌설렁탕,['17348570'],17203740,17203740,https://place.map.kakao.com/17203740,신촌설렁탕,신촌설렁탕 삼성
3869,3869,3220000-101-2011-00122,20110228,영업/정상,134.30,135822,서울특별시 강남구 논현동 121,서울 강남구 학동로 106,신사정육식당,한식,...,상수도전용,논현동,모범식당지정,신사정육식당,NaN,1390671305,1390671305,https://place.map.kakao.com/1390671305,신사정육식당,신사정육식당 논현


In [31]:
comments_last = []

for i, id in zip(last_mobeom.index, last_mobeom["id_mod2"]):
# for i, id in enumerate(df["id_mod2"][8352:8355], 8352):
    
    if id==" ":
        continue
    else:
        try:
            time.sleep(1)
            request_id = id
            url = f'https://place.map.kakao.com/main/v/{request_id}'
            response = requests.get(url)
            # print(url)

            try:

                if response.status_code == 200:
                    comment_num = 3
                    comment_data = json.loads(response.content)['comment']
                    comment_count = int(comment_data['kamapComntcnt'])
                    overall_score = int(comment_data['scoresum']) / int(comment_data['scorecnt'])
                    for c in comment_data['list']:

                        try:
                            user = c["username"]
                        except:
                            user = ''

                        try:
                            review = c['contents']
                        except:
                            review = ''

                        # comments_last.append({'점수': c['point'], '댓글': c['contents'], '작성일': c['date']})
                        comments_last.append({"name": df.loc[i, "keyword"], "user": c["username"], 
                                                'review': c['contents'], 'date': c['date'],
                                                'rate': c['point'], "avg_rate": c["userCommentAverageScore"],
                                                "review_cnt": c["userCommentCount"], "is_model": df.loc[i, "exemplary_diner"]})
                        request_comment_id = c['commentid']
                    
                    while comment_num < comment_count:
                        url = f'https://place.map.kakao.com/commentlist/v/{request_id}/{request_comment_id}'
                        response = requests.get(url)
                        if response.status_code == 200:
                            comment_data = json.loads(response.content)['comment']
                            for c in comment_data['list']:

                                try:
                                    user = c["username"]
                                except:
                                    user = ''

                                try:
                                    review = c['contents']
                                except:
                                    review = ''

                                comment_num += 1
                                try:
                                    comments_last.append({"name": df.loc[i, "keyword"], "user": c["username"], 
                                                    'review': c['contents'], 'date': c['date'],
                                                    'rate': c['point'], "avg_rate": c["userCommentAverageScore"],
                                                    "review_cnt": c["userCommentCount"], "is_model": df.loc[i, "exemplary_diner"]})
                                except:
                                    comments_last.append({"name": df.loc[i, "keyword"], "user": c["username"], 
                                                    'review': "", 'date': c['date'],
                                                    'rate': c['point'], "avg_rate": c["userCommentAverageScore"],
                                                    "review_cnt": c["userCommentCount"], "is_model": df.loc[i, "exemplary_diner"]})
                                request_comment_id = c['commentid']
                            # print(f'총 댓글 개수: {comment_count}개, 평점: {np.round(overall_score, 2)}점')
                else:
                    print('response')
                    # pass
            except:
                print('comments error')
        
        except:
            print("url_error")
            
        

comments error
response
comments error


In [34]:
reviews_3527_4176 = pd.DataFrame(comments_last)
reviews_3527_4176

,name,user,review,date,rate,avg_rate,review_cnt,is_model
0,봉추찜닭압구정로데오,참고,친절하고 자리수 많음. 맛있었고 쿠폰도 좋았습니다.,2024.04.04.,5,4.0,63,모범식당지정
1,봉추찜닭압구정로데오,정영록,진짜 너무 친절하셔서 좋아요!!,2023.11.17.,5,4.4,145,모범식당지정
2,봉추찜닭압구정로데오,🍡,매장 내부가 넓고 깨끗하고 음식도 맛있어서 좋아요,2023.08.13.,5,5.0,3,모범식당지정
3,봉추찜닭압구정로데오,나능네주코,가게도 쾌적하고 사장님도 친절하셔요 맛있게 잘 먹고가요^-^,2023.07.19.,5,4.2,214,모범식당지정
4,봉추찜닭압구정로데오,1869,친절하고 맛있어요.,2023.02.18.,5,3.0,6,모범식당지정
...,...,...,...,...,...,...,...,...
766,홍백,이현희,진짜 강추입니다ㅠㅠㅠㅠ 기본으로 나오는 선지국 맛이 일품이에요. 소고기도 상태 최상...,2019.04.18.,5,5.0,4,모범식당지정
767,홍백,찐빵콩,서비스도 많고 맛있어요 사장님 친절하심,2019.01.01.,5,5.0,6,모범식당지정
768,홍백,순례자,명성에 비해 그닥.,2018.10.03.,3,2.8,31,모범식당지정
769,홍백,강대우,점심의 육회비빔밥..,2016.09.10.,4,3.2,459,모범식당지정


In [35]:
# index 3527_4176 부터 수집한 리뷰 데이터 csv 저장 
reviews_3527_4176.to_csv("data/reviews_3527_4176.csv", index=False, encoding="utf-8")

## 여기까지

In [50]:
url

'https://place.map.kakao.com/main/v/ '

In [42]:
pd.DataFrame(comments)

""


In [51]:
reviews_df = pd.DataFrame(columns=("name", "user", 'review', 'date', 'rate', 
                                   "avg_rate", "review_cnt", "is_model"))
reviews_df

,name,user,review,date,rate,avg_rate,review_cnt,is_model


In [17]:
comments = []

for i, id in enumerate(df["id_mod2"][:1]):
    try:
        request_id = id
        url = f'https://place.map.kakao.com/main/v/{request_id}'
        response = requests.get(url)

        if response.status_code == 200:
            comment_data = json.loads(response.content)['comment']
            comment_count = int(comment_data['kamapComntcnt'])
            overall_score = int(comment_data['scoresum']) / int(comment_data['scorecnt'])
            for c in comment_data['list']:
                comments.append({'점수': c['point'], '댓글': c['contents'], '작성일': c['date']})
                request_comment_id = c['commentid']
            print(f'총 댓글 개수: {comment_count}개, 평점: {np.round(overall_score, 2)}점')
        else:
            print('error')
        
        # try:
        while len(comments) < comment_count:
            url = f'https://place.map.kakao.com/commentlist/v/{request_id}/{request_comment_id}'
            response = requests.get(url)
            if response.status_code == 200:
                comment_data = json.loads(response.content)['comment']
                for c in comment_data['list']:
                    try:
                        comments.append({"name": df.loc[i, "keyword"], "user": c["username"], 
                                        'review': c['contents'], 'date': c['date'],
                                        'rate': c['point'], "avg_rate": c["userCommentAverageScore"],
                                        "review_cnt": c["userCommentCount"], "is_model": df.loc[i, "exemplary_diner"]})
                    except:
                        comments.append({"name": df.loc[i, "keyword"], "user": c["username"], 
                                        'review': "", 'date': c['date'],
                                        'rate': c['point'], "avg_rate": c["userCommentAverageScore"],
                                        "review_cnt": c["userCommentCount"], "is_model": df.loc[i, "exemplary_diner"]})
                    request_comment_id = c['commentid']
                reviews_df = pd.concat([reviews_df, pd.DataFrame(comments)])
        #     else:
        #         # print('error in while')
        #         pass
        # except:
        #     pass        

    except:
        pass

error


In [52]:
# comments = []

# for i, id in enumerate(df["id_mod2"][:2]):
#     try:
#         request_id = id
#         url = f'https://place.map.kakao.com/main/v/{request_id}'
#         response = requests.get(url)

#         if response.status_code == 200:
#             comment_data = json.loads(response.content)['comment']
#             comment_count = int(comment_data['kamapComntcnt'])
#             overall_score = int(comment_data['scoresum']) / int(comment_data['scorecnt'])
#             for c in comment_data['list']:
#                 comments.append({'점수': c['point'], '댓글': c['contents'], '작성일': c['date']})
#                 request_comment_id = c['commentid']
#             print(f'총 댓글 개수: {comment_count}개, 평점: {np.round(overall_score, 2)}점')
#         else:
#             print('error')
#         #     for c in comment_data['list']:
#         #         try:
#         #             comments.append({"name": df.loc[i, "keyword"], "user": c["username"], 
#         #                             'review': c['contents'], 'date': c['date'],
#         #                             'rate': c['point'], "avg_rate": c["userCommentAverageScore"],
#         #                             "review_cnt": c["userCommentCount"], "is_model": df.loc[i, "exemplary_diner"]})
#         #         except:
#         #             comments.append({"name": df.loc[i, "keyword"], "user": c["username"], 
#         #                             'review': "", 'date': c['date'],
#         #                             'rate': c['point'], "avg_rate": c["userCommentAverageScore"],
#         #                             "review_cnt": c["userCommentCount"], "is_model": df.loc[i, "exemplary_diner"]})
#         #         request_comment_id = c['commentid']
#         #     # print(f'총 댓글 개수: {comment_count}개, 평점: {np.round(overall_score, 2)}점')
#         #     # reviews_df = pd.concat([reviews_df, pd.DataFrame(comments)])
#         # else:
#         #     # print('error')
#         #     pass



#         while len(comments) < comment_count:
#             url = f'https://place.map.kakao.com/commentlist/v/{request_id}/{request_comment_id}'
#             response = requests.get(url)
#             if response.status_code == 200:
#                 comment_data = json.loads(response.content)['comment']
#                 for c in comment_data['list']:
#                     try:
#                         comments.append({"name": df.loc[i, "keyword"], "user": c["username"], 
#                                         'review': c['contents'], 'date': c['date'],
#                                         'rate': c['point'], "avg_rate": c["userCommentAverageScore"],
#                                         "review_cnt": c["userCommentCount"], "is_model": df.loc[i, "exemplary_diner"]})
#                     except:
#                         comments.append({"name": df.loc[i, "keyword"], "user": c["username"], 
#                                         'review': "", 'date': c['date'],
#                                         'rate': c['point'], "avg_rate": c["userCommentAverageScore"],
#                                         "review_cnt": c["userCommentCount"], "is_model": df.loc[i, "exemplary_diner"]})
#                     request_comment_id = c['commentid']
#                 # reviews_df = pd.concat([reviews_df, pd.DataFrame(comments)])
#             else:
#                 # print('error in while')
#                 pass


#     except:
#         pass

## selenium

In [57]:
df.head()

,Unnamed: 0,MGTNO,APVPERMYMD,TRDSTATENM,SITEAREA,SITEPOSTNO,SITEWHLADDR,RDNWHLADDR,BPLCNM,UPTAENM,...,WTRSPLYFACILSENM,dong,exemplary_diner,keyword,id,id_mod,id_mod2,url,keyword2,name
0,0,3220000-101-1007-00001,19970205,영업/정상,NaN,135954,서울특별시 강남구 청담동 88-5번지,서울 강남구 도산대로55길 20,배초향,한식,...,NaN,청담동,NaN,배초향,NaN,,,,배초향,배초향 청담
1,1,3220000-101-1899-20451,19991005,영업/정상,950.66,135090,서울특별시 강남구 삼성동 159,서울 강남구 봉은사로 524,브래서리,경양식,...,상수도전용,삼성동,모범식당지정,브래서리,['10825881'],10825881,10825881,https://place.map.kakao.com10825881,브래서리,브래서리 삼성
2,2,3220000-101-1899-20453,19991005,영업/정상,473.24,135090,서울특별시 강남구 삼성동 159,서울 강남구 봉은사로 524,로비라운지,경양식,...,상수도전용,삼성동,NaN,로비라운지,['727661240'],727661240,727661240,https://place.map.kakao.com727661240,로비라운지,로비라운지 삼성
3,3,3220000-101-1899-20463,19991006,영업/정상,282.59,135090,서울특별시 강남구 삼성동 159,서울 강남구 봉은사로 524,로비바,경양식,...,상수도전용,삼성동,NaN,로비바,['21550576'],21550576,21550576,https://place.map.kakao.com21550576,로비바,로비바 삼성
4,4,3220000-101-1899-20500,19991012,영업/정상,"1,157.90",135873,서울특별시 강남구 삼성동 109-6,서울 강남구 봉은사로 637,노블발렌티,한식,...,상수도전용,삼성동,NaN,노블발렌티,"['17157328', '26597965']",17157328,17157328,https://place.map.kakao.com17157328,노블발렌티,노블발렌티 삼성


In [ ]:
# 상세보기에 조회된 모범 음식점 목록을 조회

# store_name = "솥내음 스타필드 코엑스몰"
# page_url = "https://place.map.kakao.com/710342577"


store_names = df["name"]
page_urls = df["url"]
review_results = []

driver = wb.Chrome() # 브라우저 실행

for index, (store_name, page_url) in enumerate(zip(store_names, page_urls)):

    try:

        driver.get(page_url) # 음식점 상세 화면
        time.sleep(1) # 1초 쉬기


        # total_num = int((bs(driver.page_source,'lxml')).select("span.color_g")[0].text.split(")")[0][1:])
        
        soup = bs(driver.page_source,'lxml')
        total_strong = soup.find(name="strong", attrs={"class": "total_evaluation"})
        tot_num = total_strong.find(name="span", attrs={"class": "color_b"}) # 등록된 리뷰 갯수


        more_nums = (int(tot_num.text) - 3) // 5 + 1
        print(f"--------------- {index}. {store_name} 후기 더보기 ---------------")
        for more_num in range(more_nums):
            try:
                btnMore = driver.find_element(By.XPATH, '//*[@id="mArticle"]/div[7]/div[3]/a')
                btnMore.click()
                time.sleep(1)
            except:
                btnMore = driver.find_element(By.XPATH, '//*[@id="mArticle"]/div[8]/div[3]/a')
                btnMore.click()
                time.sleep(1)


        soup = bs(driver.page_source,'lxml')

        contents_div = soup.find(name='div', attrs={"class": "evaluation_review"}) #리뷰 영역. 별점, 내용
        rates = contents_div.find_all(name="span", attrs={"class": "ico_star inner_star"}) # 별점값 목록
        reviews = contents_div.find_all(name="p", attrs={"class": "txt_comment"}) # 리뷰 목록
        users = contents_div.find_all(name="span", attrs={"class": "txt_username"})
        reviews_count = contents_div.find_all(name="span", attrs={"class": "txt_desc"})[0::2]
        avg_rates = contents_div.find_all(name="span", attrs={"class": "txt_desc"})[1::2]
        date = contents_div.find_all(name="span", attrs={"class": "time_write"})
        is_model = df.loc[index, "exemplary_diner"]

        print(f"--------------- {index}. {store_name} 리뷰 크롤링 ---------------")

        for idx in range(len(reviews)):

            # rate, review, user, review_count, avg_rate 지정
            user = users[idx].text
            rate = ((int(rates[idx].get("style")[6:-2]) / 100) * 5)    
            avg_rate = float(avg_rates[idx].text)
            review = reviews[idx].find(name="span").text
            review_count = reviews_count[idx].text
                
            # dict 파일 생성
            review_results.append({"store_name" : store_name,
                                "user" : user,                         
                                "rate" : rate,                                                    
                                "avg_rate" : avg_rate,
                                "review" : review,
                                "review_count" : review_count,
                                "date" : date,
                                "is_model" : is_model})
    
    except:
        print(f"카카오맵에 {store_name} 정보가 없음")



reviews_total_df = pd.DataFrame(review_results)

# url 채운 파일 csv로 저장
reviews_total_df.to_csv("data/reviews_total.csv", index=False, encoding="utf-8-sig")